In [19]:
import os
import re
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import colormaps as cm

import folium
from folium import plugins
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point, LineString, box
import networkx as nx
import gurobipy as gb

# Project setup

## Load data

In [20]:
# Get segment and stops on route
def get_segment_stops(
    segment_df: pd.DataFrame,
    stops_df: pd.DataFrame,
    route_id: str,
    direction_id: int = 0,
):
    segment_distance = segment_df.query(
        "route_id == @route_id & direction_id == @direction_id"
    )

    stops_in_route = stops_df[
        stops_df["stop_id"].isin(
            list(
                set(
                    segment_distance[["start_stop_id", "end_stop_id"]].values.reshape(
                        -1
                    )
                )
            )
        )
    ]

    return segment_distance, stops_in_route


# Get unique stops on route
def get_stops_on_route(
    route_ids: list, segment_df: pd.DataFrame, stops_df: pd.DataFrame
):
    route_details = {}

    for r in route_ids:
        detail = get_segment_stops(segment_df, stops_df, r, 0)

        route_details[r] = {}
        route_details[r]["segment"] = detail[0]
        route_details[r]["stops"] = detail[1]

    stops_df = pd.concat(
        [v["stops"] for k, v in route_details.items()]
    ).drop_duplicates()

    print(f"Number of total stops in routes: {len(stops_df)}")

    stops_df["geometry"] = stops_df.apply(
        lambda x: Point((float(x.stop_lon), float(x.stop_lat))), axis=1
    )

    stops_df_gpd = gpd.GeoDataFrame(
        stops_df.drop(
            columns=["location_type", "parent_station", "wheelchair_boarding"]
        ),
        geometry="geometry",
    )

    display(stops_df_gpd.head())

    return stops_df_gpd

In [21]:
segments = pd.read_csv("segments.csv")
stops = pd.read_csv("stops.csv")

segments[["route_id", "start_stop_id", "end_stop_id"]] = segments[
    ["route_id", "start_stop_id", "end_stop_id"]
].astype(str)

routes = ["24", "51", "67", "18", "105", "45", "80", "55"]

stops_df_gpd = get_stops_on_route(routes, segments, stops)

Number of total stops in routes: 328


,stop_id,stop_code,stop_name,stop_lat,stop_lon,stop_url,geometry
1769,51241,51241,Station Villa-Maria,45.479704,-73.619643,https://www.stm.info/fr/recherche#stq=51241,POINT (-73.61964 45.47970)
1806,51281,51281,Décarie / Duquette,45.478389,-73.618050,https://www.stm.info/fr/recherche#stq=51281,POINT (-73.61805 45.47839)
1848,51326,51326,Décarie / Notre-Dame-de-Grâce,45.477244,-73.615513,https://www.stm.info/fr/recherche#stq=51326,POINT (-73.61551 45.47724)
1891,51372,51372,Décarie / Côte-Saint-Antoine,45.476312,-73.613443,https://www.stm.info/fr/recherche#stq=51372,POINT (-73.61344 45.47631)
1976,51462,51462,Décarie / Sherbrooke,45.474535,-73.609490,https://www.stm.info/fr/recherche#stq=51462,POINT (-73.60949 45.47454)


#### Add depot

In [22]:
def add_depot(lat: float, lon: float, stops_df: pd.DataFrame):
    depot = pd.DataFrame(
        {
            "stop_id": ["0"],
            "stop_name": ["Depot"],
            "stop_lat": [lat],
            "stop_lon": [lon],
            "stop_code": 0.0,
        }
    )

    depot["geometry"] = Point((float(depot.stop_lon), float(depot.stop_lat)))

    stops_df_gpd = pd.concat([depot, stops_df]).reset_index(drop=True)

    return depot, stops_df_gpd

### Add Safe Zones

In [23]:
def add_depot_and_safe_zones(lat: float, lon: float, stops_df: pd.DataFrame, safe_zones_df: pd.DataFrame):
    # Define the depot
    depot = pd.DataFrame(
        {
            "stop_id": ["0"],
            "stop_name": ["Depot"],
            "stop_lat": [lat],
            "stop_lon": [lon],
            "stop_code": 0.0,
        }
    )
    depot["geometry"] = Point((float(depot.stop_lon), float(depot.stop_lat)))

    # Concatenate depot, safe zones, and stops dataframes
    stops_df_gpd = pd.concat([depot, safe_zones_df, stops_df]).reset_index(drop=True)

    return depot, stops_df_gpd


### New Random Sample with Safe Zones

In [24]:
# Sample a random subset of stops
random_stops_df_gpd = stops_df_gpd.sample(n=30, random_state=662)

# Define your safe zones DataFrame 
safe_zones_data = {
    "stop_id": ["1", "2"],
    "stop_name": ["Bell Centre - Safe Zone 1", "Olympic Stadium - Safe Zone 2"],
    "stop_lat": [45.480610, 45.561750],
    "stop_lon": [-73.565820, -73.656570],
    "stop_code": [1.0, 2.0],
}

safe_zones_df = pd.DataFrame(safe_zones_data)
safe_zones_df["geometry"] = [Point(lon, lat) for lat, lon in zip(safe_zones_df.stop_lat, safe_zones_df.stop_lon)]

# Use the modified function to add the depot and safe zones
depot_lat = 45.503256  # Depot latitude
depot_lon = -73.575901  # Depot longitude
depot, random_stops_df_gpd = add_depot_and_safe_zones(depot_lat, depot_lon, random_stops_df_gpd, safe_zones_df)

# Now 'updated_random_stops_df_gpd' includes the depot, safe zones, and the random subset of stops
print(f"Number of stops including depot and safe zones: {len(random_stops_df_gpd)}")
display(random_stops_df_gpd)


Number of stops including depot and safe zones: 33


,stop_id,stop_name,stop_lat,stop_lon,stop_code,geometry,stop_url
0,0,Depot,45.503256,-73.575901,0.0,POINT (-73.575901 45.503256),NaN
1,1,Bell Centre - Safe Zone 1,45.480610,-73.565820,1.0,POINT (-73.56582 45.48061),NaN
2,2,Olympic Stadium - Safe Zone 2,45.561750,-73.656570,2.0,POINT (-73.65657 45.56175),NaN
3,50983,Papineau / François-H.-Prévost,45.557554,-73.628413,50983.0,POINT (-73.628413 45.557554),https://www.stm.info/fr/recherche#stq=50983
4,50422,Saint-Laurent / de Port-Royal,45.546222,-73.656965,50422.0,POINT (-73.656965 45.546222),https://www.stm.info/fr/recherche#stq=50422
5,51029,Sherbrooke / Cavendish,45.463498,-73.629216,51029.0,POINT (-73.629216 45.463498),https://www.stm.info/fr/recherche#stq=51029
6,52603,Beaubien / Lacordaire,45.580168,-73.565258,52603.0,POINT (-73.565258 45.580168),https://www.stm.info/fr/recherche#stq=52603
7,52795,Beaubien / François-Boivin,45.590246,-73.559273,52795.0,POINT (-73.559273 45.590246),https://www.stm.info/fr/recherche#stq=52795
8,51750,du Parc / Saint-Joseph,45.518983,-73.594163,51750.0,POINT (-73.594163 45.518983),https://www.stm.info/fr/recherche#stq=51750
9,51069,Ogilvy / Querbes,45.529197,-73.625268,51069.0,POINT (-73.625268 45.529197),https://www.stm.info/fr/recherche#stq=51069


## Calculate distance matrix

In [25]:
# G = ox.graph_from_place("Montreal, Canada", network_type="drive")
G = ox.load_graphml("montreal_drive.graphml")

In [26]:
### run if the stops change

In [27]:

print(f"Number of stops: {len(random_stops_df_gpd)}")

distance_matrix = np.zeros((len(random_stops_df_gpd), len(random_stops_df_gpd)))
for i, stop1 in enumerate(random_stops_df_gpd.itertuples()):
    print(f"Calculating distance for stop {i}")
    for j in range(i + 1, len(random_stops_df_gpd)):
        stop2 = random_stops_df_gpd.iloc[j]

        origin = ox.nearest_nodes(G, stop1.stop_lon, stop1.stop_lat)
        destination = ox.nearest_nodes(G, stop2.stop_lon, stop2.stop_lat)

        try:
            distance = nx.shortest_path_length(G, origin, destination, weight="length")
        except nx.NetworkXNoPath:
            distance = np.Inf

        distance_matrix[i, j] = distance
        distance_matrix[j, i] = distance

    print("-" * 100)

distance_matrix = pd.DataFrame(
    distance_matrix / 1000,
    columns=random_stops_df_gpd.stop_id,
    index=random_stops_df_gpd.stop_id,
)

distance_matrix.to_json("distance_matrix_extension.json")

Number of stops: 33
Calculating distance for stop 0
----------------------------------------------------------------------------------------------------
Calculating distance for stop 1
----------------------------------------------------------------------------------------------------
Calculating distance for stop 2
----------------------------------------------------------------------------------------------------
Calculating distance for stop 3
----------------------------------------------------------------------------------------------------
Calculating distance for stop 4
----------------------------------------------------------------------------------------------------
Calculating distance for stop 5
----------------------------------------------------------------------------------------------------
Calculating distance for stop 6
----------------------------------------------------------------------------------------------------
Calculating distance for stop 7
-----------------

In [34]:
def get_distance_matrix(stops_df: pd.DataFrame, G: nx.Graph):
    if os.path.exists("distance_matrix_extension.json"):
        print("Loading distance matrix from JSON")
        distance_matrix = pd.read_json("distance_matrix_extension.json")

        distance_matrix.columns = distance_matrix.columns.astype(str)
        distance_matrix.index = distance_matrix.index.astype(str)

        if len(distance_matrix) != len(stops_df):
            print("⚠️ Distance matrix does not match number of stops")
            print(f"Number of stops in distance matrix: {len(distance_matrix)}")
            print(f"Number of stops in DataFrame: {len(stops_df)}")
        else:
            print("✅ Distance matrix loaded successfully")
            print(f"Number of stops in distance matrix: {len(distance_matrix)}")
            print(f"Number of stops in DataFrame: {len(stops_df)}")

        return distance_matrix

    print(f"Calculating distance for {len(stops_df)} stops")

    distance_matrix = np.zeros((len(stops_df), len(stops_df)))

    for i, stop1 in enumerate(stops_df.itertuples()):
        print(f"Calculating distance for stop {i}")
        for j in range(i + 1, len(stops_df)):
            stop2 = stops_df.iloc[j]

            origin = ox.nearest_nodes(G, stop1.stop_lon, stop1.stop_lat)
            destination = ox.nearest_nodes(G, stop2.stop_lon, stop2.stop_lat)

            try:
                distance = nx.shortest_path_length(
                    G, origin, destination, weight="length"
                )
            except nx.NetworkXNoPath:
                distance = np.Inf

            distance_matrix[i, j] = distance
            distance_matrix[j, i] = distance

        print("-" * 50)

    # Convert to km and save as JSON
    distance_matrix = pd.DataFrame(
        distance_matrix / 1000,
        columns=stops_df.stop_id,
        index=stops_df.stop_id,
    )

    distance_matrix.to_json("distance_matrix_extension.json")

    return distance_matrix

In [35]:
distance_matrix = get_distance_matrix(random_stops_df_gpd, G)

Loading distance matrix from JSON
✅ Distance matrix loaded successfully
Number of stops in distance matrix: 33
Number of stops in DataFrame: 33


## Add disaster area

In [36]:
def add_disaster_area(stops_df: pd.DataFrame, disaster_bounds: list):
    disaster_area = box(*disaster_bounds)

    stops_df_gpd = gpd.GeoDataFrame(stops_df, geometry="geometry")

    stops_in_disaster_area = stops_df_gpd[stops_df_gpd.within(disaster_area)]

    print(f"Number of stops in disaster area: {len(stops_in_disaster_area)}")

    display(stops_in_disaster_area)

    return stops_in_disaster_area, disaster_area


stops_in_disaster_area, disaster_area = add_disaster_area(
    random_stops_df_gpd, [-73.682213, 45.535972, -73.604622, 45.572994]
)

Number of stops in disaster area: 5


,stop_id,stop_name,stop_lat,stop_lon,stop_code,geometry,stop_url
2,2,Olympic Stadium - Safe Zone 2,45.561750,-73.656570,2.0,POINT (-73.65657 45.56175),NaN
3,50983,Papineau / François-H.-Prévost,45.557554,-73.628413,50983.0,POINT (-73.62841 45.55755),https://www.stm.info/fr/recherche#stq=50983
4,50422,Saint-Laurent / de Port-Royal,45.546222,-73.656965,50422.0,POINT (-73.65696 45.54622),https://www.stm.info/fr/recherche#stq=50422
20,50303,Station Henri-Bourassa,45.554262,-73.668563,50303.0,POINT (-73.66856 45.55426),https://www.stm.info/fr/recherche#stq=50303
29,50431,Papineau / Prieur,45.569139,-73.655548,50431.0,POINT (-73.65555 45.56914),https://www.stm.info/fr/recherche#stq=50431


## Find Closest Stops to disaster Area

In [37]:
def find_closest_stops_outside_disaster(stops_df: pd.DataFrame, disaster_bounds: list, num_closest_stops: int = 5):
    """
    Identify the closest bus stops outside the disaster area.

    :param stops_df: DataFrame containing bus stops information.
    :param disaster_bounds: List of coordinates defining the disaster area bounds.
    :param num_closest_stops: Number of closest stops to find.
    :return: DataFrame of closest stops outside the disaster area.
    """
    # Create a polygon for the disaster area
    disaster_area = box(*disaster_bounds)

    # Convert stops DataFrame to GeoDataFrame
    stops_df_gpd = gpd.GeoDataFrame(stops_df, geometry=gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat))

    # Exclude stops within the disaster area
    stops_outside_disaster = stops_df_gpd[~stops_df_gpd.within(disaster_area)]

    # Calculate the distance of each stop from the disaster area
    stops_outside_disaster['distance_to_disaster'] = stops_outside_disaster['geometry'].apply(lambda x: x.distance(disaster_area))

    # Sort stops by distance and select the closest ones outside the disaster area
    closest_stops = stops_outside_disaster.sort_values(by='distance_to_disaster').head(num_closest_stops)

    print(f"Closest {num_closest_stops} stops to the disaster area, outside of it:")
    display(closest_stops)

    return closest_stops

# Example usage
closest_stops_to_disaster = find_closest_stops_outside_disaster(
    stops_df=random_stops_df_gpd, 
    disaster_bounds=[-73.50826, 45.57889, -73.60963, 45.70587],
    num_closest_stops=3
)


Closest 3 stops to the disaster area, outside of it:


c:\Users\Lakshya Agarwal\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,stop_id,stop_name,stop_lat,stop_lon,stop_code,geometry,stop_url,distance_to_disaster
3,50983,Papineau / François-H.-Prévost,45.557554,-73.628413,50983.0,POINT (-73.62841 45.55755),https://www.stm.info/fr/recherche#stq=50983,0.028426
31,52520,Saint-Michel / Saint-Joseph,45.549551,-73.567988,52520.0,POINT (-73.56799 45.54955),https://www.stm.info/fr/recherche#stq=52520,0.029339
14,51781,Beaubien / des Écores,45.547472,-73.592774,51781.0,POINT (-73.59277 45.54747),https://www.stm.info/fr/recherche#stq=51781,0.031418


## View stops in sample

In [38]:
stops_map = folium.Map(
    location=[45.5048542, -73.5691235],
    zoom_start=11,
    tiles="cartodbpositron",
    width="100%",
)

# Add disaster area
folium.GeoJson(
    disaster_area,
    name="Disaster area",
    style_function=lambda x: {
        "color": "#ff0000",
        "fillColor": "#ff0000",
        "weight": 1,
        "fillOpacity": 0.4,
    },
).add_to(stops_map)

# Add stops in network
for stop in random_stops_df_gpd.itertuples():
    folium.CircleMarker(
        location=[stop.stop_lat, stop.stop_lon],
        radius=5,
        color=(
            "red"
            if stop.stop_id in stops_in_disaster_area.stop_id.values.tolist()
            else "darkgreen"
        ),
        fill=True,
        fill_opacity=1,
        fill_color=(
            "red"
            if stop.stop_id in stops_in_disaster_area.stop_id.values.tolist()
            else "darkgreen"
        ),
        tooltip=f"{stop.stop_name} ({stop.stop_id})",
        popup=f"""
        <div>
            <h4>{stop.stop_name} ({stop.stop_id})</h4>
            <h4>Distance from depot: {distance_matrix.loc["0", stop.stop_id]:.1f} km</h4>
        </div>
        """,
    ).add_to(stops_map)

    
# Add safe zone stops in a different color
for stop in safe_zones_df.itertuples():
    folium.CircleMarker(
        location=[stop.stop_lat, stop.stop_lon],
        radius=5,
        color="blue",  # Unique color for safe zone stops
        fill=True,
        fill_opacity=1,
        fill_color="blue",  # Matching fill color
        tooltip=f"{stop.stop_name} ({stop.stop_id})",
        popup=f"""
        <div>
            <h4>{stop.stop_name} ({stop.stop_id})</h4>
            <h4>Distance from depot: {distance_matrix.loc["0", stop.stop_id]:.1f} km</h4>
        </div>
        """,
    ).add_to(stops_map)

    
# Add Depot in a different colour

for stop in random_stops_df_gpd.itertuples():
    # Check if the stop is the depot
    if stop.stop_name.lower() == "depot":
       folium.CircleMarker(
        location=[stop.stop_lat, stop.stop_lon],
        radius=5,
        color="black",  # Unique color for safe zone stops
        fill=True,
        fill_opacity=1,
        fill_color="black",  # Matching fill color
        tooltip=f"{stop.stop_name} ({stop.stop_id})",
        popup=f"""
        <div>
            <h4>{stop.stop_name} ({stop.stop_id})</h4>
            <h4>Distance from depot: {distance_matrix.loc["0", stop.stop_id]:.1f} km</h4>
        </div>
        """,
    ).add_to(stops_map) 

folium.plugins.Fullscreen(position="topright").add_to(stops_map)
folium.plugins.MousePosition(position="topright").add_to(stops_map)

stops_map

# MILP Model - Split delivery vehicle routing problem

## Parameters

In [41]:
rng = np.random.default_rng(5)

num_buses = 10
BUS_CAPACITY = 75

distance_matrix_model = distance_matrix.drop(
    columns=stops_in_disaster_area.stop_id, index=stops_in_disaster_area.stop_id
)

stops = list(distance_matrix_model.columns)
num_stops = len(stops)

distance_matrix_model = distance_matrix_model.loc[stops, stops]

demand = {stop: rng.integers(0, 100) for stop in stops}
demand[stops[0]] = 0


print(f"Total demand: {sum(demand.values())}")
print(f"Total capacity: {num_buses * BUS_CAPACITY}")

Total demand: 1124
Total capacity: 750


### New Parameters with Demand aggregation

In [64]:
rng = np.random.default_rng(5)

num_buses = 20
BUS_CAPACITY = 100

stops = list(distance_matrix_model.columns)
num_stops = len(stops)

distance_matrix_model = distance_matrix_model.loc[stops, stops]

demand = {
    "0": 0,
    "50727": 137,
    "50986": 27,
    "51029": 112,
    "51032": 57,
    "51069": 72,
    "51241": 40,
    "51372": 80,
    "51741": 19,
    "51748": 88,
    "51750": 66,
    "51781": 39,
    "51904": 91,
    "51921": 139,
    "52277": 40,
    "52459": 21,
    "52511": 37,
    "52520": 61,
    "52603": 4,
    "52610": 1,
    "52637": 8,
    "52647": 7,
    "52710": 7,
    "52795": 113,
    "53677": 33,
    "53751": 54,
    "56199": 105,
}

print(f"Total demand: {sum(demand.values())}")
print(f"Total capacity: {num_buses * BUS_CAPACITY}")

Total demand: 1458
Total capacity: 2000


In [65]:
# Function to find the closest stop outside the disaster area
def find_closest_stop(stop_id, distance_matrix, exclude_stops):
    distances = distance_matrix[stop_id]
    distances = distances.drop(index=exclude_stops + [stop_id])
    return distances.idxmin()

# Aggregate demand from disaster stops to nearest stops
for disaster_stop in stops_in_disaster_area['stop_id']:
    closest_stop = find_closest_stop(disaster_stop, distance_matrix, stops_in_disaster_area['stop_id'].tolist())
    demand[closest_stop] += demand.get(disaster_stop, 0)
    demand[disaster_stop] = 0  # Optional: Set disaster stop demand to 0

# Verify the updated demand
print("Updated demand after aggregation:", sum(demand.values()))


Updated demand after aggregation: 1458


In [66]:
### drop disaster area stops

distance_matrix_model = distance_matrix.drop(
    columns=stops_in_disaster_area.stop_id, index=stops_in_disaster_area.stop_id
)

## Split nodes

### Split methods

#### Geometric progression

$$
\begin{align*}
D_{ix} &= \frac{2^{x-1}}{\sum_{i=1}^{S} 2^{x-1}} D_{i}
\end{align*}
$$

$D_{ix}$ is rounded down to the nearest integer. If $\sum_{i=1}^{S} D_{ix} < D_{i}$, then $D_{iS} = D_{iS} + D_{i} - \sum_{i=1}^{S} D_{ix}$.

In [67]:
# Split the node demand into smaller demands according to a geometric progression
def split_demand_geometric(node_demand, BUS_CAPACITY, fraction, fraction_sum):
    demands = [np.floor(node_demand * (f / fraction_sum)) for f in fraction]
    demands = list(filter(lambda x: x > 0, demands))

    if sum(demands) < node_demand:
        demands[-1] += node_demand - sum(demands)

    i = len(demands) - 1

    while demands[i] >= BUS_CAPACITY * 1:
        new_demand = split_demand_geometric(
            demands[i], BUS_CAPACITY, fraction, fraction_sum
        )
        demands.pop(i)
        demands.extend(new_demand)
        i -= 1

    return demands

#### Capacity-based split

$$
\begin{align*}
D_{ix} &= Q &\quad \forall x \in \{1, \dots, S = \frac{q_{i}}{Q}\} \\
\end{align*}
$$

$D_{ix}$ is rounded down to the nearest integer. If $\sum_{i=1}^{S} D_{ix} < D_{i}$, then $D_{i(S+1)} = D_{i} - \sum_{i=1}^{S} D_{ix}$.


In [68]:
# Split the node demand into smaller demands according to BUS_CAPACITY
def split_demand_capacity(node_demand, BUS_CAPACITY):
    demands = [BUS_CAPACITY for _ in range(int(np.floor(node_demand / BUS_CAPACITY)))]
    demands.append(node_demand - sum(demands))

    return demands

#### Equal split

$$
\begin{align*}
D_{ix} &= 1 &\quad \forall x \in \{1, \dots, q_{i}\} \\
\end{align*}
$$


In [69]:
# Split the node demand into equal demand nodes of 1
def split_demand_equal(node_demand, BUS_CAPACITY):
    demands = [1 for _ in range(node_demand)]

    return demands

#### Random split

$$
\begin{align*}
D_{ix} &= Z \sim \{1, Q\}
\end{align*}
$$


In [70]:
# Split the node demand into random demand values

def split_demand_random(node_demand, BUS_CAPACITY, rng: np.random.Generator):
    demands = []
    remaining_demand = node_demand

    while remaining_demand > 0:
        demand = rng.integers(1, BUS_CAPACITY)
        demands.append(demand)
        remaining_demand -= demand

    if sum(demands) > node_demand:
        demands[-1] -= sum(demands) - node_demand

    return demands

#### Split individual demand nodes

In [71]:
def split_demand_node(demand, node, BUS_CAPACITY, rng, split_type: str = "geometric"):
    node_demand = demand[node]
    if split_type == "geometric":
        S = 100
        fraction = [2 ** (i - 1) for i in range(1, S + 1)]
        fraction_sum = sum(fraction)
        demands = split_demand_geometric(
            node_demand, BUS_CAPACITY, fraction, fraction_sum
        )
    elif split_type == "capacity":
        demands = split_demand_capacity(node_demand, BUS_CAPACITY)
    elif split_type == "equal":
        demands = split_demand_equal(node_demand, BUS_CAPACITY)
    elif split_type == "random":
        demands = split_demand_random(node_demand, BUS_CAPACITY, rng)
    else:
        raise ValueError("Invalid demand split type")

    return {f"{node}_{i}": d for i, d in enumerate(demands, 1)}


### Reconstruct parameters

In [72]:
def reconstruct_demand(demand, BUS_CAPACITY, rng, split_type: str = "geometric"):
    nodes_exceeding_demand = {}
    new_demand = demand.copy()
    for k, v in demand.items():
        if v > BUS_CAPACITY:
            new_nodes = split_demand_node(demand, k, BUS_CAPACITY, rng, split_type)

            new_demand.pop(k)
            new_demand.update(new_nodes)

            nodes_exceeding_demand[k] = new_nodes

    print(f"New nodes added for: {list(nodes_exceeding_demand.keys())}")

    return new_demand, nodes_exceeding_demand

In [73]:
# Update distance matrix to include split nodes. Distance between split nodes of same parent node is 0
def update_distance_matrix(distance_matrix, nodes_exceeding_demand, new_demand):
    distance_matrix_model = distance_matrix.copy()

    for node in nodes_exceeding_demand:
        for i in range(1, len([k for k in new_demand.keys() if node in k]) + 1):
            distance_matrix_model[f"{node}_{i}"] = distance_matrix_model[node]
            distance_matrix_model.loc[f"{node}_{i}"] = distance_matrix_model.loc[node]
            distance_matrix_model.loc[f"{node}_{i}", node] = 0

        distance_matrix_model.drop(columns=[node], index=[node], inplace=True)

    return distance_matrix_model

In [75]:
safe_zones = ['1', '2']

# Loop to remove specified stop IDs from the demand dictionary
for zone in safe_zones:
    demand[zone] = 0

In [76]:
new_demand, nodes_exceeding_demand = reconstruct_demand(demand, BUS_CAPACITY, "geometric")
distance_matrix_model_pruned = update_distance_matrix(
    distance_matrix_model, nodes_exceeding_demand, new_demand
)

New nodes added for: ['50727', '51029', '51921', '52795', '56199']


In [77]:
for k, v in new_demand.items():
    assert v <= BUS_CAPACITY

assert sum(new_demand.values()) == sum(demand.values())

In [78]:
new_stops = list(distance_matrix_model_pruned.columns)
num_new_stops = len(new_stops)

print(f"Total number of stops (overall): {num_stops}")
print(f"Total number of stops (including splits): {num_new_stops}")

Total number of stops (overall): 28
Total number of stops (including splits): 55


In [79]:
new_demand

{'0': 0,
 '50986': 27,
 '51032': 57,
 '51069': 72,
 '51241': 40,
 '51372': 80,
 '51741': 19,
 '51748': 88,
 '51750': 66,
 '51781': 39,
 '51904': 91,
 '52277': 40,
 '52459': 21,
 '52511': 37,
 '52520': 61,
 '52603': 4,
 '52610': 1,
 '52637': 8,
 '52647': 7,
 '52710': 7,
 '53677': 33,
 '53751': 54,
 '2': 0,
 '50983': 0,
 '50422': 0,
 '50303': 0,
 '50431': 0,
 '1': 0,
 '50727_1': 1.0,
 '50727_2': 2.0,
 '50727_3': 4.0,
 '50727_4': 8.0,
 '50727_5': 17.0,
 '50727_6': 34.0,
 '50727_7': 71.0,
 '51029_1': 1.0,
 '51029_2': 3.0,
 '51029_3': 7.0,
 '51029_4': 14.0,
 '51029_5': 28.0,
 '51029_6': 59.0,
 '51921_1': 1.0,
 '51921_2': 2.0,
 '51921_3': 4.0,
 '51921_4': 8.0,
 '51921_5': 17.0,
 '51921_6': 34.0,
 '51921_7': 73.0,
 '52795_1': 1.0,
 '52795_2': 3.0,
 '52795_3': 7.0,
 '52795_4': 14.0,
 '52795_5': 28.0,
 '52795_6': 60.0,
 '56199_1': 1.0,
 '56199_2': 3.0,
 '56199_3': 6.0,
 '56199_4': 13.0,
 '56199_5': 26.0,
 '56199_6': 56.0}

In [80]:
len(new_demand)

60

In [81]:
# Final consistency check
for stop in new_stops:
    if stop not in new_demand:
        print(f"Missing demand value for stop {stop}")


## Model

Revised model includes the additional safezone constraint and the nearest stop and disaster stop parameters

### New Model With Binary variable y

In [82]:
def define_model(
    stops: list,
    buses: int,
    demand: dict,
    distance_matrix: pd.DataFrame,
    BUS_CAPACITY: int,
    nearest_stops: list,  # Add nearest stops parameter
    disaster_stops: list,# Add disaster stops parameter
    safe_zones: list, # add safe zones
    DISTANCE_THRESHOLD: float = 5.0,
    **params,
):
    # ----------------------------------------------------------------------------------------------
    # Model

    model = gb.Model("Bus Routing")
    model.Params.MIPGap = params.get("MIPGap", 0.05)
    model.Params.TimeLimit = params.get("TimeLimit", 60 * 3)
    model.Params.MIPFocus = params.get("MIPFocus", 1)
    model.Params.LogToConsole = params.get("LogToConsole", 1)

    # ----------------------------------------------------------------------------------------------
    # Decision Variables

    x = model.addVars(
        stops,
        stops,
        buses,
        vtype=gb.GRB.BINARY,
        name=(
            f"{i} -> {j} (bus {k})" for i in stops for j in stops for k in range(buses)
        ),
    )

    u = model.addVars(
        stops,
        vtype=gb.GRB.INTEGER,
        name=(f"Load at Stop {i}" for i in stops),
    )

    # ----------------------------------------------------------------------------------------------
    # Objective Function
    model.setObjective(
        gb.quicksum(
            distance_matrix.loc[i, j] * x[i, j, k]
            for i in stops
            for j in stops
            for k in range(buses)
        ),
        gb.GRB.MINIMIZE,
    )

    # ----------------------------------------------------------------------------------------------
    # Constraints

    # Vehicle leaves nodes that it enters
    model.addConstrs(
        (
            gb.quicksum(x[j, i, k] for j in stops)
            == gb.quicksum(x[i, j, k] for j in stops)
            for i in stops
            for k in range(buses)
        ),
        name="Vehicle leaves nodes that it enters",
    )

    # Every node is entered once
    model.addConstrs(
        (
            gb.quicksum(x[i, j, k] for i in stops for k in range(buses)) == 1
            for j in stops[1:]
        ),
        name="Every node is entered once",
    )

    # Every vehicle leaves the depot
    model.addConstrs(
        (gb.quicksum(x[stops[0], j, k] for j in stops[1:]) <= 1 for k in range(buses)),
        name="Every vehicle may leave the depot if needed",
    )

    # Capacity constraint
    model.addConstrs(
        (
            gb.quicksum(demand[j] * x[i, j, k] for j in stops[1:] for i in stops)
            <= BUS_CAPACITY
            for k in range(buses)
        ),
        name="Capacity constraint",
    )

    # No travel between same node
    model.addConstrs(
        (x[i, i, k] == 0 for i in stops for k in range(buses)),
        name="No same node",
    )

    # Subtour elimination constraints
    model.addConstrs(
        (
            u[j] - u[i] >= demand[j] - BUS_CAPACITY * (1 - x[i, j, k])
            for i in stops[1:]
            for j in stops[1:]
            for k in range(buses)
            if i != j
        ),
        name="Subtour elimination constraint",
    )

    model.addConstrs(
        (u[i] >= demand[i] for i in stops[1:]),
        name="Lower bound for u",
    )

    model.addConstrs(
        (u[i] <= BUS_CAPACITY for i in stops[1:]),
        name="Upper bound for u",
    )

    # Distance between two travel nodes is less than specified distance
    model.addConstrs(
        (
            distance_matrix.loc[i, j] * x[i, j, k] <= DISTANCE_THRESHOLD
            for i in stops[1:]
            for j in stops[1:]
            for k in range(buses)
        ),
        name="Distance between two travel nodes is less than a specified distance",
    ) 
    
    # Add binary variables for visiting nearest stops
    y = model.addVars(nearest_stops, range(num_buses), vtype=gb.GRB.BINARY, name="y")

    # Print all keys in x for debugging
    #print("All keys in x:", x.keys())

    # Debugging: Check for missing keys in the link_y constraint
    for k in range(num_buses):
        for n in nearest_stops:
            key = (n, '0', k)  # Assuming '0' is the depot
            if key not in x:
                print(f"Missing key in x: {key}")
            else:
                model.addConstr(y[n, k] <= sum(x[n, j, k] for j in stops), name=f"link_y_{n}_{k}")

    for k in range(num_buses):
        for n in nearest_stops:
            # Modify the constraint to include a valid stop_j for every stop_i in the safe_zones
            model.addConstr(
                sum(x[n, j, k] for j in stops if (n, j, k) in x) >= y[n, k],
                name=f"nearest_to_safe_zone_{n}_{k}"
            )





    # ----------------------------------------------------------------------------------------------
    # Solve model
    model._vars = x
    model.update()

    return model

### New Model Call

In [83]:
model = define_model(
    new_stops,  # List of all stops
    num_buses,  # Number of buses
    new_demand,  # Demand dictionary
    distance_matrix_model_pruned,  # Distance matrix DataFrame
    BUS_CAPACITY,  # Bus capacity
    nearest_stops=[stop.stop_id for stop in closest_stops_to_disaster.itertuples()],  # Nearest stops to disaster
    disaster_stops=[stop.stop_id for stop in stops_in_disaster_area.itertuples()],  # Stops in disaster area
    safe_zones=['1','2'],  
    DISTANCE_THRESHOLD=7.0,  # Default or specific value for distance threshold
    MIPGap=0.1,  # Other parameters
    TimeLimit=60 * 15,
    MIPFocus=1,
    LogToConsole=1
) 
model.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-30
Set parameter MIPGap to value 0.1
Set parameter TimeLimit to value 900
Set parameter MIPFocus to value 1
Missing key in x: ('50983', '0', 0)
Missing key in x: ('50983', '0', 1)
Missing key in x: ('50983', '0', 2)
Missing key in x: ('50983', '0', 3)
Missing key in x: ('50983', '0', 4)
Missing key in x: ('50983', '0', 5)
Missing key in x: ('50983', '0', 6)
Missing key in x: ('50983', '0', 7)
Missing key in x: ('50983', '0', 8)
Missing key in x: ('50983', '0', 9)
Missing key in x: ('50983', '0', 10)
Missing key in x: ('50983', '0', 11)
Missing key in x: ('50983', '0', 12)
Missing key in x: ('50983', '0', 13)
Missing key in x: ('50983', '0', 14)
Missing key in x: ('50983', '0', 15)
Missing key in x: ('50983', '0', 16)
Missing key in x: ('50983', '0', 17)
Missing key in x: ('50983', '0', 18)
Missing key in x: ('50983', '0', 19)
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\Lakshya Agarwal\AppData\Roaming\Python\Python310\site-packages\ipykernel\iostream.py", line 384, in write
    def write(self, string):
KeyboardInterrupt: 


   997   927   44.69726   16  243  220.18854   41.21434  81.3%   131  136s


In [1]:
closest_stops_to_disaster

NameError: name 'closest_stops_to_disaster' is not defined

In [37]:
stops_in_disaster_area

,stop_id,stop_name,stop_lat,stop_lon,stop_code,geometry,stop_url
4,54638,du Val d'Anjou / de la Nantaise,45.597701,-73.553117,54638.0,POINT (-73.55312 45.59770),https://www.stm.info/fr/recherche#stq=54638
21,54958,Langelier / Jarry,45.595085,-73.581845,54958.0,POINT (-73.58185 45.59508),https://www.stm.info/fr/recherche#stq=54958
22,54618,Beaubien / des Galeries-d'Anjou,45.594933,-73.556338,54618.0,POINT (-73.55634 45.59493),https://www.stm.info/fr/recherche#stq=54618


## Solution

### Optimal routes

### Third Show solution Function to deal with errors (final)

In [38]:
def show_solution(model, stops, buses, demand, nodes_exceeding_demand, distance_matrix, stops_df):
    x = model._vars

    print('-'*100)
    print(f"Objective value: {model.objVal:.2f} km")
    print('-'*100)

    # Distance by bus
    distance_bus = pd.DataFrame(
        {
            "bus": k,
            "distance": sum(
                distance_matrix.loc[i, j] * x[i, j, k].x for i in stops for j in stops
            ),
        }
        for k in range(buses)
    )

    distance_bus = distance_bus[distance_bus["distance"] > 0].reset_index(drop=True)

    # Bus paths
    bus_path = {}
    for k in range(buses):
        bus_path[k] = []
        for i in stops:
            for j in stops:
                if x[i, j, k].x == 1:
                    bus_path[k].append(
                        {
                            "start_stop": i,
                            "end_stop": j,
                            "distance": distance_matrix.loc[i, j],
                        }
                    )

    # Convert to dataframe with bus route number
    bus_path_df = []
    for k, v in bus_path.items():
        bus_path_df.append(pd.DataFrame(v).assign(bus=k))

    bus_path_df = pd.concat(bus_path_df)

    # Paths for each bus
    paths = {}
    grouped = bus_path_df.groupby("bus")

    # Iterate over each bus group
    for bus, group in grouped:
        sorted_group = group.sort_values(by=["start_stop", "end_stop"]).reset_index(drop=True)
        path = ["0"]  # Initialize the path with the depot

        # Start with the first stop after the depot
        current_stop_rows = sorted_group.loc[sorted_group["start_stop"] == "0", "end_stop"]
        if not current_stop_rows.empty:
            current_stop = current_stop_rows.values[0]
            path.append(current_stop)

            # Follow the chain of stops
            while True:
                next_stop_rows = sorted_group.loc[sorted_group["start_stop"] == current_stop, "end_stop"]
                if next_stop_rows.empty:
                    break  # If there's no next stop, complete the path
                next_stop = next_stop_rows.values[0]

                if next_stop == "0":
                    break  # If the next stop is the depot, complete the path
                path.append(next_stop)
                current_stop = next_stop

            paths[bus] = path

    # Calculate steps for buses with valid paths
    bus_path_df["step"] = bus_path_df.apply(
        lambda x: paths[x.bus].index(x.start_stop) if x.bus in paths else None, axis=1
    )

    bus_path_df.sort_values(by=["bus", "step"], inplace=True)

    bus_path_df["demand"] = bus_path_df["end_stop"].map(demand)

    bus_path_df[["start_stop", "end_stop"]] = bus_path_df[
        ["start_stop", "end_stop"]
    ].applymap(lambda x: x.split("_")[0])

    bus_path_df = bus_path_df.merge(
        stops_df[["stop_id", "stop_name", "geometry"]],
        left_on="end_stop",
        right_on="stop_id",
        how="left",
    )

    bus_path_df["step_demand"] = bus_path_df.groupby("bus")["demand"].cumsum()

    bus_path_df["is_split"] = bus_path_df["stop_id"].isin(nodes_exceeding_demand.keys())

    num_buses_used = bus_path_df.bus.nunique()
    print(f"Number of buses used: {num_buses_used}")

    # Filter out rows for buses without valid paths
    bus_path_df = bus_path_df.dropna(subset=["step"])

    return distance_bus, bus_path_df, paths


In [39]:
def plot_networkx_bus(bus_path_df):
    plots = {}
    for bus in bus_path_df.bus.unique():
        # print(f"Bus {bus + 1}:")

        bus_route = bus_path_df[bus_path_df["bus"] == bus].reset_index(drop=True)

        fig = plt.figure(figsize=(20, 10))

        g = nx.DiGraph()

        for segment in bus_route.itertuples():
            g.add_edge(
                segment.start_stop,
                segment.end_stop,
                weight=segment.distance,
                step=segment.step,
                demands={"demand": segment.demand, "load": segment.step_demand},
                label=f"{segment.start_stop}-{segment.end_stop}",
            )

        pos = nx.circular_layout(g)

        nx.draw_networkx(g, pos, with_labels=True, node_size=500, node_color="skyblue")
        nx.draw_networkx_edge_labels(
            g, pos, edge_labels=nx.get_edge_attributes(g, "demands")
        )

        plots[bus] = fig

    plt.close("all")
    return plots


#network_plots = plot_networkx_bus(bus_path_df)

In [40]:
def build_route_df(bus_path_df, depot):
    routes_gdf = (
        gpd.GeoDataFrame(bus_path_df.groupby("bus")["geometry"].apply(list))
        .rename(columns={"points": "geometry"})
        .reset_index()
    )

    # add depot to each geometry
    routes_gdf["points"] = routes_gdf.apply(
        lambda x: [Point((float(depot.stop_lon), float(depot.stop_lat)))] + x.geometry,
        axis=1,
    )

    routes_gdf["geometry"] = routes_gdf["points"].apply(LineString)
    routes_gdf.drop(columns=["points"], inplace=True)

    routes_gdf = routes_gdf.merge(
        bus_path_df.groupby("bus")["demand"].sum().rename("demand").reset_index(),
        on="bus",
    )

    routes_gdf = gpd.GeoDataFrame(routes_gdf, geometry="geometry")
    routes_gdf.crs = "EPSG:4326"

    # display(routes_gdf)

    return routes_gdf

### Map the routes

In [50]:
def plot_routes(distance_matrix, bus_path_df, routes_gdf, num_buses, split_type,safe_zones_df):
    colormap_route = [
        mcolors.rgb2hex(c) for c in list(plt.cm.rainbow(np.linspace(0, 1, num_buses)))
    ]

    route_map = folium.Map(
        location=[45.5048542, -73.5691235],
        zoom_start=11,
        tiles="cartodbpositron",
        width="100%",
    )

    folium.GeoJson(
        disaster_area,
        name="Disaster area",
        style_function=lambda x: {
            "color": "#ff0000",
            "fillColor": "#ff0000",
            "weight": 1,
            "fillOpacity": 0.3,
        },
    ).add_to(route_map)

    
     # Add markers for safe zones
    for safe_zone in safe_zones_df.itertuples():
        folium.Marker(
            location=[safe_zone.stop_lat, safe_zone.stop_lon],
            icon=folium.Icon(color="green", icon="info-sign"),
            popup=f"<b>{safe_zone.stop_name}</b>",
            tooltip=safe_zone.stop_name
        ).add_to(route_map)
    
    
    for stop in bus_path_df.itertuples():
        folium.CircleMarker(
            location=[stop.geometry.coords[0][1], stop.geometry.coords[0][0]],
            radius=5,
            color=(
                colormap_route[stop.bus]
                if not stop.is_split
                else "purple"
                if stop.stop_id != "0"
                else "black"
            ),
            fill=True,
            fill_opacity=1,
            fill_color=(
                colormap_route[stop.bus]
                if not stop.is_split
                else "purple"
                if stop.stop_id != "0"
                else "black"
            ),
            tooltip=f"""
            <b>{stop.stop_name} ({stop.stop_id})</b>
            <br>
            Route: {bus_path_df[bus_path_df["stop_id"] == stop.stop_id]['bus'].values[0] + 1}
            <br>
            Step: {bus_path_df[bus_path_df["stop_id"] == stop.stop_id]['step'].values[0] + 1}
            <br>
            Demand: {bus_path_df[bus_path_df["stop_id"] == stop.stop_id]['demand'].values[0]}
            <br>
            Load: {bus_path_df[bus_path_df["stop_id"] == stop.stop_id]['step_demand'].values[0]}
            <br>
            Has split demand?: {stop.is_split}
            """,
            popup=f"""
            <div>
                <h4>{stop.stop_name} ({stop.stop_id})</h4>
                <h4>Distance from depot: {distance_matrix.loc["0", stop.stop_id]:.1f} km</h4>
            </div>
            """,
        ).add_to(route_map)

    for stop in stops_in_disaster_area.itertuples():
        folium.CircleMarker(
            location=[stop.geometry.coords[0][1], stop.geometry.coords[0][0]],
            radius=5,
            color="red",
            fill=True,
            fill_opacity=1,
            fill_color="red",
            tooltip=f"""
            <b>{stop.stop_name} ({stop.stop_id})</b>
            """,
            popup=f"""
            <div>
                <h4>{stop.stop_name} ({stop.stop_id})</h4>
                <h4>Distance from depot: {distance_matrix.loc["0", stop.stop_id]:.1f} km</h4>
            </div>
            """,
        ).add_to(route_map)

    for route in routes_gdf.itertuples():
        route_layer = folium.FeatureGroup(f"Route {route.bus + 1}")
        folium.PolyLine(
            locations=[(p[1], p[0]) for p in route.geometry.coords],
            color=colormap_route[route.bus],
            weight=3,
            opacity=0.6,
            tooltip=f"Route {route.bus + 1}",
            popup=f"""
            <div>
                <h5>Route {route.bus + 1}</h5>
                <h5>Total demand: {route.demand}</h5>
            </div>
            """,
        ).add_to(route_layer)

        route_layer.add_to(route_map)

    folium.plugins.Fullscreen(position="topright").add_to(route_map)
    folium.plugins.MousePosition(position="topright").add_to(route_map)
    folium.LayerControl().add_to(route_map)

    route_map.save(f"route_map_split_{split_type}.html")
    return route_map

# MILP model as a function

### 3rd MILP Model with debugging and demand aggregation (final)

In [52]:
def solve_model(
    depot,
    stops_df,
    distance_matrix,
    num_buses,
    BUS_CAPACITY,
    nearest_stops: list,  # Add nearest stops parameter
    disaster_stops: list,  # Add disaster stops parameter
    safe_zones: list,
    DEMAND_LIMIT: int = 100,
    DISTANCE_THRESHOLD: float = 5.0,
    split_type: str = "geometric",
    MIPGap: float = 0.2,
    TimeLimit: int = 60 * 3,
    MIPFocus: int = 1,
    LogToConsole: int = 1,
):
    rng = np.random.default_rng(5)


    stops = list(distance_matrix.columns)
    num_stops = len(stops)

    distance_matrix_model = distance_matrix.loc[stops, stops]

    demand = {stop: rng.integers(1, DEMAND_LIMIT) for stop in stops}
    demand[stops[0]] = 0
    
    for zone in safe_zones:
        demand[zone] = 0
    
    # Debugging: Print the demand before and after aggregation
    print("Demand before aggregation:", demand)
    
    # Aggregate disaster stop demand to the nearest stops
    for disaster_stop in disaster_stops:
        closest_stop = find_closest_stop(disaster_stop, distance_matrix, disaster_stops)
        if closest_stop in demand:  # Ensuring closest stop is in demand dictionary
            demand[closest_stop] += demand.get(disaster_stop, 0)
        else:
            print(f"Closest stop not found for disaster stop {disaster_stop}")

    # Update the distance matrix after demand aggregation
    distance_matrix_model = distance_matrix.drop(
        columns=disaster_stops, index=disaster_stops
    )
    
    # Also, remove the disaster stops from the demand dictionary
    for disaster_stop in disaster_stops:
        demand.pop(disaster_stop, None) 
    
    print("Demand after aggregation:", demand)
    
    demand[stops[0]] = 0
    
    for zone in safe_zones:
        demand[zone] = 0
    
    # Update stops list and check for consistency
    stops = list(distance_matrix_model.columns)
    if set(stops) != set(demand.keys()):
        print("Mismatch in stops and demand keys!")
        print("Stops not in demand:", set(stops) - set(demand.keys()))
        print("Demand keys not in stops:", set(demand.keys()) - set(stops))
    
    num_stops = len(stops)
        
    print('-'*100)
    print(f"Total demand: {sum(demand.values())}")
    print(f"Total capacity: {num_buses * BUS_CAPACITY}")
    print('-'*100)

    new_demand, nodes_exceeding_demand = reconstruct_demand(
        demand, BUS_CAPACITY, rng, split_type
    )

    distance_matrix_model_pruned = update_distance_matrix(
        distance_matrix_model, nodes_exceeding_demand, new_demand
    )

    for k, v in new_demand.items():
        assert v <= BUS_CAPACITY

    assert sum(new_demand.values()) == sum(demand.values())

    new_stops = list(distance_matrix_model_pruned.columns)
    num_new_stops = len(new_stops)

    print('-'*100)
    print(f"Total number of stops: {num_stops}")
    print(f"Total number of new stops (including splits): {num_new_stops}")
    print('-'*100)

    model = define_model(
        new_stops,
        num_buses,
        new_demand,
        distance_matrix_model_pruned,
        BUS_CAPACITY,
        DISTANCE_THRESHOLD=DISTANCE_THRESHOLD,
        nearest_stops=[stop.stop_id for stop in closest_stops_to_disaster.itertuples()],  # Nearest stops to disaster
        disaster_stops=[stop.stop_id for stop in stops_in_disaster_area.itertuples()],  # Pass disaster stops
        safe_zones = ['1','2'],
        MIPGap=MIPGap,
        TimeLimit=TimeLimit,
        MIPFocus=MIPFocus,
        LogToConsole=LogToConsole,
    )
    
    print('-'*100)
    print(f'Solving model with split type "{split_type.capitalize()}"')
    print('-'*100)

    model.optimize()

    distance_bus, bus_path_df, paths = show_solution(
        model,
        new_stops,
        num_buses,
        new_demand,
        nodes_exceeding_demand,
        distance_matrix_model_pruned,
        stops_df,
    )

    network_plots = plot_networkx_bus(bus_path_df)

    routes_gdf = build_route_df(bus_path_df, depot)

    route_map = plot_routes(
        distance_matrix, bus_path_df, routes_gdf, num_buses, split_type,safe_zones_df
    )

    return {
        "split_type": split_type,
        "demand": demand,
        "new_demand": new_demand,
        "model": model,
        "distance_bus": distance_bus,
        "bus_path_df": bus_path_df,
        "paths": paths,
        "network_plots": network_plots,
        "routes_gdf": routes_gdf,
        "route_map": route_map,
    }

In [43]:
NUM_BUSES = 15
BUS_CAPACITY = 80
DEMAND_LIMIT = 120
TIME_LIMIT = 60 * 3
LOG_TO_CONSOLE = 1

In [44]:
nearest_stops=[stop.stop_id for stop in closest_stops_to_disaster.itertuples()],  # Nearest stops to disaster
disaster_stops=[stop.stop_id for stop in stops_in_disaster_area.itertuples()],  #

nearest_stops 
disaster_stops

(['54638', '54958', '54618'],)

### New Geometric Split

In [56]:
geometric_split_model = solve_model(
    depot=depot,  # Depot location
    stops_df=random_stops_df_gpd,  # DataFrame of stops
    distance_matrix=distance_matrix,  # Distance matrix
    num_buses=NUM_BUSES,  # Number of buses
    BUS_CAPACITY=BUS_CAPACITY,  # Bus capacity
    nearest_stops=[stop.stop_id for stop in closest_stops_to_disaster.itertuples()],
    disaster_stops=[stop.stop_id for stop in stops_in_disaster_area.itertuples()],
    safe_zones = ['1','2'],
    DEMAND_LIMIT=DEMAND_LIMIT,  # Demand limit per stop
    split_type="geometric",  # Type of demand split
    TimeLimit=TIME_LIMIT,  # Time limit for optimization
    LogToConsole=LOG_TO_CONSOLE,  # Logging flag
)

Demand before aggregation: {'0': 0, '1': 0, '2': 0, '52718': 97, '54638': 56, '51511': 62, '50516': 75, '53677': 35, '56274': 117, '55333': 7, '50110': 34, '50828': 46, '51983': 68, '52895': 49, '55278': 16, '50716': 6, '52075': 1, '51388': 6, '50662': 18, '50576': 119, '52510': 23, '54958': 78, '54618': 90}
Demand after aggregation: {'0': 0, '1': 0, '2': 0, '52718': 97, '51511': 62, '50516': 75, '53677': 35, '56274': 117, '55333': 7, '50110': 34, '50828': 46, '51983': 68, '52895': 49, '55278': 94, '50716': 6, '52075': 147, '51388': 6, '50662': 18, '50576': 119, '52510': 23}
----------------------------------------------------------------------------------------------------
Total demand: 1003
Total capacity: 1200
----------------------------------------------------------------------------------------------------
New nodes added for: ['52718', '56274', '55278', '52075', '50576']
----------------------------------------------------------------------------------------------------
Total nu

Number of buses used: 15


### Third new Capacity Split - Aggregated demand

In [53]:
capacity_split_model= solve_model(
    depot=depot,  # Depot location
    stops_df=random_stops_df_gpd,  # DataFrame of stops
    distance_matrix=distance_matrix,  # Distance matrix
    num_buses=NUM_BUSES,  # Number of buses
    BUS_CAPACITY=BUS_CAPACITY,  # Bus capacity
    nearest_stops=[stop.stop_id for stop in closest_stops_to_disaster.itertuples()],
    disaster_stops=[stop.stop_id for stop in stops_in_disaster_area.itertuples()],
    safe_zones = ['1','2'],
    DEMAND_LIMIT=DEMAND_LIMIT,  # Demand limit per stop
    split_type="capacity",  # Type of demand split
    TimeLimit=TIME_LIMIT,  # Time limit for optimization
    LogToConsole=LOG_TO_CONSOLE,  # Logging flag
)

Demand before aggregation: {'0': 0, '1': 0, '2': 0, '52718': 97, '54638': 56, '51511': 62, '50516': 75, '53677': 35, '56274': 117, '55333': 7, '50110': 34, '50828': 46, '51983': 68, '52895': 49, '55278': 16, '50716': 6, '52075': 1, '51388': 6, '50662': 18, '50576': 119, '52510': 23, '54958': 78, '54618': 90}
Demand after aggregation: {'0': 0, '1': 0, '2': 0, '52718': 97, '51511': 62, '50516': 75, '53677': 35, '56274': 117, '55333': 7, '50110': 34, '50828': 46, '51983': 68, '52895': 49, '55278': 94, '50716': 6, '52075': 147, '51388': 6, '50662': 18, '50576': 119, '52510': 23}
----------------------------------------------------------------------------------------------------
Total demand: 1003
Total capacity: 1200
----------------------------------------------------------------------------------------------------
New nodes added for: ['52718', '56274', '55278', '52075', '50576']
----------------------------------------------------------------------------------------------------
Total nu

### New Random Split - aggregated demand

In [57]:
random_split_model = solve_model(
    depot=depot,  # Depot location
    stops_df=random_stops_df_gpd,  # DataFrame of stops
    distance_matrix=distance_matrix,  # Distance matrix
    num_buses=NUM_BUSES,  # Number of buses
    BUS_CAPACITY=BUS_CAPACITY,  # Bus capacity
    nearest_stops=[stop.stop_id for stop in closest_stops_to_disaster.itertuples()],
    disaster_stops=[stop.stop_id for stop in stops_in_disaster_area.itertuples()],
    safe_zones = ['1','2'],
    DEMAND_LIMIT=DEMAND_LIMIT,  # Demand limit per stop
    split_type="random",  # Type of demand split
    TimeLimit=TIME_LIMIT,  # Time limit for optimization
    LogToConsole=LOG_TO_CONSOLE,  # Logging flag
)

Demand before aggregation: {'0': 0, '1': 0, '2': 0, '52718': 97, '54638': 56, '51511': 62, '50516': 75, '53677': 35, '56274': 117, '55333': 7, '50110': 34, '50828': 46, '51983': 68, '52895': 49, '55278': 16, '50716': 6, '52075': 1, '51388': 6, '50662': 18, '50576': 119, '52510': 23, '54958': 78, '54618': 90}
Demand after aggregation: {'0': 0, '1': 0, '2': 0, '52718': 97, '51511': 62, '50516': 75, '53677': 35, '56274': 117, '55333': 7, '50110': 34, '50828': 46, '51983': 68, '52895': 49, '55278': 94, '50716': 6, '52075': 147, '51388': 6, '50662': 18, '50576': 119, '52510': 23}
----------------------------------------------------------------------------------------------------
Total demand: 1003
Total capacity: 1200
----------------------------------------------------------------------------------------------------
New nodes added for: ['52718', '56274', '55278', '52075', '50576']
----------------------------------------------------------------------------------------------------
Total nu

  Learned: 6
  Cover: 19
  Implied bound: 43
  Clique: 304
  MIR: 75
  StrongCG: 39
  Flow cover: 84
  GUB cover: 12
  Inf proof: 4
  Zero half: 7
  RLT: 223
  Relax-and-lift: 39

Explored 60231 nodes (4341440 simplex iterations) in 180.12 seconds
Thread count was 8 (of 8 available processors)

Solution count 9: 240.461 240.461 240.461 ... 258.401

Time limit reached
Best objective 2.404611160000e+02, best bound 1.566532585686e+02, gap 34.8530%
----------------------------------------------------------------------------------------------------
Objective value: 240.46 km
----------------------------------------------------------------------------------------------------
Number of buses used: 15


# Compare models

In [48]:
models = [geometric_split_model, capacity_split_model, random_split_model] 


model_bus_routes = []
for model_name, model in zip(['geometric', 'capacity', 'random'], models):
    bus_routes_df = model["bus_path_df"].copy()
    bus_routes_df["MODEL_TYPE"] = model_name
    model_bus_routes.append(bus_routes_df)

model_bus_routes_df = pd.concat(model_bus_routes)

model_bus_routes_df

,start_stop,end_stop,distance,bus,step,demand,stop_id,stop_name,geometry,step_demand,is_split,MODEL_TYPE
0,0,56274,5.577917,0,0.0,3.0,56274,Sherbrooke / Metcalfe,POINT (-73.59759 45.48394),3.0,True,geometric
1,56274,56274,0.000000,0,1.0,7.0,56274,Sherbrooke / Metcalfe,POINT (-73.59759 45.48394),10.0,True,geometric
2,56274,56274,0.000000,0,2.0,29.0,56274,Sherbrooke / Metcalfe,POINT (-73.59759 45.48394),39.0,True,geometric
3,56274,56274,0.000000,0,3.0,1.0,56274,Sherbrooke / Metcalfe,POINT (-73.59759 45.48394),40.0,True,geometric
4,56274,1,2.884043,0,4.0,0.0,1,Bell Centre - Safe Zone 1,POINT (-73.56582 45.48061),40.0,False,geometric
...,...,...,...,...,...,...,...,...,...,...,...,...
42,0,51983,7.033586,13,0.0,68.0,51983,Beaubien / 13e Avenue,POINT (-73.58513 45.55626),68.0,False,random
43,51983,0,7.033586,13,1.0,0.0,0,Depot,POINT (-73.58009 45.50964),68.0,False,random
44,0,53677,4.253418,14,0.0,35.0,53677,Queen-Mary / du Frère-André (Oratoire St-Joseph),POINT (-73.62043 45.49330),35.0,False,random
45,53677,51388,2.366429,14,1.0,6.0,51388,Station Édouard-Montpetit,POINT (-73.61307 45.50956),41.0,False,random


In [49]:
# Compare distance travelled, number of buses used, number of split nodes by split_type
model_bus_routes_summary = model_bus_routes_df.groupby("MODEL_TYPE").agg(
    {
        "distance": "sum",
        "bus": "nunique",
        "stop_id": lambda x: len(x.unique()),
        "is_split": "sum",
    }
).rename(
    columns={
        "distance": "Total distance (km)",
        "bus": "Number of buses",
        "stop_id": "Number of stops",
        "is_split": "Number of split nodes",
    }
)

# Add number of 1-stop trips
model_bus_routes_summary = model_bus_routes_summary.merge(
    (
        model_bus_routes_df.groupby(["MODEL_TYPE", "bus"])
        .agg({"step": "max"})
        .query("step == 1")
        .groupby("MODEL_TYPE")
        .size()
        .to_frame()
        .rename(columns={0: "1-stop trips"})
    ),
    left_index=True,
    right_index=True,
)

model_bus_routes_summary.style.background_gradient(
    cmap="GnBu", axis=0
)

,Total distance (km),Number of buses,Number of stops,Number of split nodes,1-stop trips
MODEL_TYPE,,,,,
capacity,239.129006,15,20,10,9
geometric,208.411436,14,18,28,2
random,240.461116,15,20,18,5


## Safe Zone Formulation

Let $N = \{n_1, n_2, \ldots, n_r\}$ be the set of nearest stops to the flooded area.

For every bus $k \in K$, if it visits any stop in $O$, it must first visit at least one stop in $N$ before going to any stop in $O$ or returning to the depot:
$$\sum_{i \in N} \sum_{j \in O} x_{ijk} \geq \sum_{j \in O} x_{j0k} \quad \forall k \in K$$
